In [1]:
import os
import logging
import requests
from bs4 import BeautifulSoup
import logging
from linkedin_jobs_scraper import LinkedinScraper
from linkedin_jobs_scraper.events import Events, EventData, EventMetrics
from linkedin_jobs_scraper.query import Query, QueryOptions, QueryFilters
from linkedin_jobs_scraper.filters import RelevanceFilters, TimeFilters, TypeFilters, ExperienceLevelFilters, \
    OnSiteOrRemoteFilters, SalaryBaseFilters
import pandas as pd
from linkedin_jobs_scraper.utils.chrome_driver import build_driver
import time
from datetime import datetime
import csv

In [2]:
FieldName = ['job_id', 'title', 'company','date','link','insight','description']
today = datetime.now().strftime("%y_%m_%d")
job_scan_csv = f"job_scan_{today}.csv"
tracking_csv_route = f"execute_tracking_{today}.csv"


if os.path.exists(job_scan_csv):
    execute_tracking = pd.read_csv(tracking_csv_route)
else:
    linked_page = pd.read_csv('linkedin_pages.csv')
    execute_tracking = linked_page.dropna()
    execute_tracking['Status'] = 'Not Processed'

# Create the output csv if not exist
if job_scan_csv and not os.path.exists(job_scan_csv):
    with open(job_scan_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file,fieldnames=FieldName)
        writer.writeheader()


C:\Users\10125\AppData\Local\Temp\ipykernel_28568\2132437514.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  execute_tracking['Status'] = 'Not Processed'


In [3]:
# Function to append each result to the CSV file
def write_output(result, csv_file,FieldName = FieldName):
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=FieldName)
        writer.writerow(result)


# Change root logger level (default is WARN)
logging.basicConfig(level=logging.INFO)


# Fired once for each successfully processed job
def on_data(data: EventData):
    print('Get ', data.company)
    output = {'job_id':data.job_id, 'title':data.title, 'company':data.company,'date':data.date,'link':data.link,'insight':data.insights,'description':data.description}
    write_output(output, job_scan_csv)

# Fired once for each page (25 jobs)
def on_metrics(metrics: EventMetrics):
    print('[ON_METRICS]', str(metrics))


def on_error(error):
    print('[ON_ERROR]', error)


def on_end():
    print('[ON_END]')


scraper = LinkedinScraper(
    chrome_executable_path=None,  # Custom Chrome executable path (e.g. /foo/bar/bin/chromedriver)
    chrome_binary_location=None,  
    chrome_options=None,  # Custom Chrome options here
    headless=True,  # Overrides headless mode only if chrome_options is None
    max_workers=1,  # How many threads will be spawned to run queries concurrently (one Chrome driver for each thread)
    slow_mo=0.5,  # Slow down the scraper to avoid 'Too many requests 429' errors (in seconds)
    page_load_timeout=40,
  # Page load timeout (in seconds)    
)

# Add event listeners
scraper.on(Events.DATA, on_data)
scraper.on(Events.ERROR, on_error)
scraper.on(Events.END, on_end)


INFO:li:scraper:('Using strategy AuthenticatedStrategy',)


In [4]:

for index, row in execute_tracking.iterrows():
    if execute_tracking.at[index, 'Status'] == 'Processed':
        continue
    link =row['job_link']
    query = Query(    
        options=QueryOptions(
            # locations=['United States'],
            skip_promoted_jobs=True,
            limit = 1000,      
            filters=QueryFilters(
                company_jobs_url= link
            )
        )
    )
    scraper.run(query)
    execute_tracking.at[index, 'Status'] = 'Processed'
    execute_tracking.to_csv(tracking_csv_route, index=False)

INFO:li:scraper:('Starting new query', "Query(query= options=QueryOptions(limit=1000 locations=['Worldwide'] filters=QueryFilters(company_jobs_url=https://www.linkedin.com/jobs/search/?f_C=6640471&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3971570718%2C3973575366) apply_link=False skip_promoted_jobs=True page_offset=0))")


https://www.linkedin.com/jobs/search?f_C=6640471&geoId=92000000&f_WT=&start=0


INFO:li:scraper:('Chrome debugger url', 'http://localhost:61733')
INFO:li:scraper:('Websocket debugger url: ', 'ws://localhost:61733/devtools/page/C6068B5DDCB9B0A0EE7D1E90CDA0F93E')
INFO:li:scraper:('[][Worldwide]', 'Setting authentication cookie')
INFO:li:scraper:('[][Worldwide]', 'Opening https://www.linkedin.com/jobs/search?f_C=6640471&geoId=92000000&start=0')
INFO:li:scraper:('[][Worldwide]', 'Session is valid')
INFO:li:scraper:('[][Worldwide][1]', 'Processed')


Get  Solera Health


INFO:li:scraper:('[][Worldwide][2]', 'Processed')


...

INFO:li:scraper:('[][Worldwide][69]', 'Processed')


Get  Rendr


INFO:li:scraper:('[][Worldwide][70]', 'Processed')


Get  Rendr


INFO:li:scraper:('[][Worldwide][71]', 'Skipped because promoted')
INFO:li:scraper:('[][Worldwide][72]', 'Skipped because promoted')
INFO:li:scraper:('[][Worldwide][73]', 'Skipped because promoted')
INFO:li:scraper:('[][Worldwide]', 'No more jobs to process in this page')
INFO:li:scraper:('[][Worldwide]', 'Metrics:', '{ processed: 70, failed: 0, missed: 2, skipped: 3 }')
INFO:li:scraper:('[][Worldwide]', 'Pagination requested [3]')
INFO:li:scraper:('[][Worldwide]', 'Opening https://www.linkedin.com/jobs/search?f_C=42080325&geoId=92000000&start=75')
INFO:li:scraper:('[][Worldwide]', 'Waiting for new jobs to load')
INFO:li:scraper:('[][Worldwide]', "Couldn't find more jobs for the running query")
